In [1]:
# These packages are pre-installed on Google Colab, but are included here to facilitate running this notebook locally
!pip install --quiet matplotlib
!pip install --quiet scikit-learn
!pip install --quiet numpy
!pip install --quiet scipy
!pip install --quiet pacmap
!pip install --quiet leidenalg
!pip install --quiet sinfo
# snRNA-seq analysis
!pip install --quiet scanpy
!pip install --quiet omnipath
!pip install --quiet decoupler

In [ ]:
!git clone https://github.com/EugOT/CN-pr-MDD-snRNA-seq.git
%cd /content/CN-pr-MDD-snRNA-seq/

In [ ]:
import os 
import random
import pacmap
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import decoupler as dc

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
from anndata.experimental.multi_files import AnnCollection

In [ ]:
matplotlib.rcParams["pdf.use14corefonts"] = True
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
matplotlib.rcParams["font.family"] = "sans-serif"
matplotlib.rcParams["font.sans-serif"] = ["Helvetica"]
matplotlib.rcParams["figure.max_open_warning"] = 20000

reseed = 42
random.seed(reseed)
np.random.seed(reseed)

sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)
# sc.settings.figdir = PLOTS_DIR
sc.settings.set_figure_params(
    dpi=180, dpi_save=600, vector_friendly=True, format="pdf", transparent=True
)
sc.settings.autoshow = True
sc.settings.autosave = False

In [ ]:
samples_males = pd.read_csv("data/PRJNA602867.tsv", delimiter="\t")
samples_males = samples_males[["Run", "Condition", "LibraryName", "BioProject", "Sex", "NTotalCells"]]

samples_females = pd.read_csv("data/PRJNA883411.tsv", delimiter="\t")
samples_females = samples_females[["Run", "Condition", "LibraryName", "BioProject", "Sex", "NTotalCells"]]

In [ ]:
filtering_parameters = {
    "hvg_top_genes":6000, #orig: 5000
    "pearson_residuals_top_genes":3000, # orig: 2000
    "min_cells_per_gene":5, #orig: 10
    "min_genes_per_cell":200, #orig: 500
}

In [ ]:
males = sc.read_h5ad(
    "data/PRJNA602867-whole_dataset-fpr_0.001-clusters.h5ad"
)
males.obs['Run'] = males.obs['orig.ident']
males.obs = pd.merge(samples_males, males.obs, on="Run").set_index("cell_name", drop=False)
males.uns["name"] = "PRJNA602867"
sc.pp.filter_cells(males, min_genes=filtering_parameters["min_genes_per_cell"])
sc.pp.filter_genes(males, min_cells=filtering_parameters["min_cells_per_gene"])

females = sc.read_h5ad(
    "data/PRJNA883411-whole_dataset-fpr_0.001-clusters.h5ad"
)
females.obs['Run'] = females.obs['orig.ident']
females.obs = pd.merge(females.obs, samples_females, on="Run", how = "inner").set_index("cell_name", drop=False)
females.uns["name"] = "PRJNA883411"
sc.pp.filter_cells(females, min_genes=filtering_parameters["min_genes_per_cell"])
sc.pp.filter_genes(females, min_cells=filtering_parameters["min_cells_per_gene"])

In [ ]:
males.obs = males.obs[[
    'cell_name',
    'background_fraction',
    'droplet_efficiency',
    'doublet_score',
    'nFeature_Diff',
    'nCount_Diff',
    'percent_mito',
    'percent_ribo',
    'percent_mito_ribo',
    'percent_hb',
    'log10GenesPerUMI',
    'k_tree',
    'Run',
    'Condition',
    'BioProject',
    'Sex',
    'n_genes']]

females.obs = females.obs[[
    'cell_name',
    'background_fraction',
    'droplet_efficiency',
    'doublet_score',
    'nFeature_Diff',
    'nCount_Diff',
    'percent_mito',
    'percent_ribo',
    'percent_mito_ribo',
    'percent_hb',
    'log10GenesPerUMI',
    'k_tree',
    'Run',
    'Condition',
    'BioProject',
    'Sex',
    'n_genes']]

In [ ]:
# join males and females datasets into a single AnnData object
adata = ad.concat([males, females], join="inner")

# save raw counts and normalized counts for later use
adata.layers["raw"] = adata.X.copy()
adata.layers["sqrt_norm"] = np.sqrt(sc.pp.normalize_total(adata, inplace=False)["X"])
adata.raw = adata

# apply the Pearson residuals recipe 
sc.experimental.pp.recipe_pearson_residuals(adata, n_top_genes=2000, batch_key="Run")


In [ ]:
adata

In [ ]:
#Filter neurons based on the three gene markers for PV Interneurons

pvalb_gene_index = adata.var_names.get_loc('PVALB')
gad1_gene_index = adata.var_names.get_loc('GAD1')
gad2_gene_index = adata.var_names.get_loc('GAD2')
pv_neurons_mask = (adata.X[:, pvalb_gene_index].toarray() > 0) & (adata.X[:, gad1_gene_index].toarray() > 0) & (adata.X[:, gad2_gene_index].toarray() > 0)
only_pv_neurons = adata[pv_neurons_mask]


In [ ]:
#number of available cells per condition
adata.obs.groupby(['Condition'])['cell_name'].count().reset_index()


In [ ]:
#number of available cells per sex
adata.obs.groupby(['Sex'])['cell_name'].count().reset_index()

In [ ]:
adata

In [ ]:
adata = only_pv_neurons.copy()

# identify highly variable genes using Pearson residuals method
sc.experimental.pp.highly_variable_genes(adata, flavor="pearson_residuals", n_top_genes=filtering_parameters["hvg_top_genes"])

#only include highly variable genes in data
adata = adata[:, adata.var["highly_variable"]]

# combine Sex and k_tree into k_rtee
adata.obs["k_tree"] = adata.obs["Sex"].astype(str) + "_" + adata.obs["k_tree"].astype(str)


In [ ]:
#keep raw and depth-normalized counts for later
adata.layers["raw"] = adata.X.copy()
adata.layers["sqrt_norm"] = np.sqrt(sc.pp.normalize_total(adata, inplace=False)["X"])
adata.X = adata.X.toarray() + 1e-6
adata.raw = adata

#perform normalization with recipe_pearson_residuals
sc.experimental.pp.recipe_pearson_residuals(adata, n_top_genes=filtering_parameters["pearson_residuals_top_genes"], batch_key="Run")

In [ ]:
#extract highly variable genes and perform dimensionality reduction
hvgs = adata.var["highly_variable"]
embedding = pacmap.PaCMAP(
    n_components=2, n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0, apply_pca=False
)
adata.obsm["X_pacmap"] = embedding.fit_transform(adata.obsm["X_pca"], init="pca")
n_cells = len(adata)

#get nearest neighbours for umapping
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=50, method='umap')
sc.tl.umap(adata, method='umap')
sc.tl.leiden(adata)

In [ ]:
#Trying to find DEGs for either condition

adata.obs['ConditionCategory'] = adata.obs['Condition'].astype('category')
sc.tl.rank_genes_groups(
    adata,
    groupby="ConditionCategory",  # Column defining your groups
    method="t-test",      # You can also try 'logreg', 'wilcoxon', or 't-test_overestim_var'
    key_added="degs_condition"
)

In [ ]:
sc.pl.rank_genes_groups(adata, key='degs_condition')

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='logreg', key_added="leiden")

In [ ]:
sc.pl.rank_genes_groups(adata, key='leiden')

In [ ]:
adata

In [ ]:
adata.obs.groupby(["leiden","Condition"])["cell_name"].count().reset_index()